In [2]:
import torch

In [3]:
from util.data import get_dl, get_train_ds, get_val_ds
from util.training_utils import train, check_accuracy, save, load
from models.Base3DUNet import Base3DUNet
from util.loss import DiceLoss, BCEDiceLoss

In [4]:
BATCH_SIZE = 4
EPOCHS = 100
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
LR = 0.0001
# DEVICE = 'cpu'

In [6]:
train_dl = get_dl(get_train_ds(), BATCH_SIZE,nw=2)
val_dl = get_dl(get_val_ds(), BATCH_SIZE, nw=2)

images: 221, masks: 221 
images: 148, masks: 148 


In [9]:
import gc
torch.cuda.empty_cache()
gc.collect()

6

In [10]:
model = Base3DUNet(3, 3, features=[64, 128, 256, 512], up_sample=True).to(DEVICE)
print(model)

Base3DUNet(
  (pooling): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (downs): ModuleList(
    (0): DoubleConv3D(
      (conv): Sequential(
        (0): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (1): DoubleConv3D(
      (conv): Sequential(
        (0): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1),

In [11]:
# load(model, "weights/3D/3d_50e_adam_b4_dice")

In [12]:
# model.train()

In [13]:

print(f"total parameters = {sum(p.numel() for p in model.parameters())}")
print(f"total learnable parameters = {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

total parameters = 94130947
total learnable parameters = 94130947


In [14]:
opt = torch.optim.Adam(model.parameters(), lr=LR)
# loss = torch.nn.BCEWithLogitsLoss # cannot be used because there's a lot of imbalance anyway, so it is better to combine it with dice
# loss = BCEDiceLoss()
loss = DiceLoss()

In [15]:
print(DEVICE)


cuda


In [ ]:
train(model, epochs=EPOCHS, training_loader=train_dl, loss_fn=loss, device=DEVICE, optimizer=opt)

At epoch [58/100]:  50%|█████     | 28/56 [01:13<01:11,  2.54s/it, loss=0.146]

In [ ]:
check_accuracy(val_dl,model,DEVICE)
# convT
# 100 DICE Results: Results: 115510440/116391936 with accuracy 99.3155 Dice score: 0.7647337317466736
# 100 BCE-DICE Results: 115626234/116391936 with accuracy 99.3421 Dice score: 0.7765376567840576
# upsample
# 100 DICE Results: Results: 115510440/116391936 with accuracy A Dice score: D
# 100 BCE-DICE Results: 115626234/116391936 with accuracy A Dice score: D

In [ ]:
# saving sample
save(model,"weights/3D/3d_100e_adam_b4_dice_upsampler")
print("saved the model...")

In [ ]:
save(model,"weights/3D/3d_100e_adam_b4_dice_upsampler")

In [14]:
train(model, epochs=EPOCHS, training_loader=train_dl, loss_fn=loss, device=DEVICE, optimizer=opt)

At epoch [10/50]:  12%|█▎        | 7/56 [00:14<01:28,  1.81s/it, loss=0.128]

In [15]:
check_accuracy(val_dl,model,DEVICE)

In [19]:
# loading sample
model1 = Base3DUNet(3,3)
load(model,"weights/3D/t1")